# Safari Edits and Editors

From [T292103](https://phabricator.wikimedia.org/T292103)

**Editors:** Of the unique non-bot editors who have published at least one successful edit in the past 90 days, what percentages of them have published at least one with Safari on mobile? With Safari on desktop? 

**Edits:** Of the successful non-bot edits that have been made in the past 90 days, what percentage of them were made with Safari on mobile? With Safari on desktop? 

We'd like both of these grouped by wiki project (e.g. English Wikipedia, Indonesian Wikisource, Commons, etc…), country (e.g. United States, Indonesia, etc…), and user tenure bucket (e.g. less than 1 day, 1–7 days, etc…).

In [38]:
import os
import re
import json
import datetime as dt

import numpy as np
import pandas as pd
import wmfdata

from wmfdata import hive, spark, mariadb, presto

import ipaddress

import findspark

In [5]:
SPARK_HOME = os.environ.get("SPARK_HOME", "/usr/lib/spark2")
findspark.init(SPARK_HOME)
from pyspark.sql import functions as F, types as T, Window

In [6]:
spark_session = spark.get_session(type="yarn-large")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


## EditAttemptStep Queries

We'd like to count the number of unique registered editors, number of unique IP editors, number of edits, and also count those using Safari, and distinguish between desktop and mobile.

It's easier to think about this from a per-user perspective. For each user, how many edits did they make, how many were on Safari, how many were on desktop Safari, how many were on mobile Safari?

We'll for now ignore the country and user tenur bucket parts of this analysis.

In [32]:
presto.run('''
SELECT
    DISTINCT user_agent_map['os_family']
FROM event.editattemptstep
WHERE event.action = 'saveSuccess'
AND event.is_oversample = false -- Only use sampled events
AND regexp_like(user_agent_map['browser_family'], 'Safari')
''')

,_col0
0,Linux
1,Android
2,Windows
3,Fedora
4,FreeBSD
5,Mac OS X
6,Other
7,iOS


In [92]:
presto.run('''
SELECT
    DISTINCT user_agent_map['browser_family']
FROM event.editattemptstep
WHERE event.action = 'saveSuccess'
AND event.is_oversample = false -- Only use sampled events
AND regexp_like(user_agent_map['browser_family'], 'Safari')
''')

,_col0
0,Mobile Safari UI/WKWebView
1,Mobile Safari
2,Safari


These who do not identify as Mac OS X or iOS is why we'll have to limit the count of "Safari editors" to those editing on two specific OSes.

In [55]:
edits_editors_query = '''
WITH reg_editors AS (
    SELECT
        wiki,
        event.user_id,
        COUNT(1) AS num_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] IN ('Mac OS X', 'iOS'), 1, NULL)) AS num_safari_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'Mac OS X', 1, NULL)) AS num_safari_macos_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'iOS', 1, NULL)) AS num_safari_ios_edits
    FROM event.editattemptstep
    WHERE event.action = 'saveSuccess'
    AND event.is_oversample = false -- Only use sampled events
    AND event.user_id != 0 -- No IP edits
    AND event.user_class IS NULL -- Set to "bot" if a bot user
    GROUP BY wiki, event.user_id
),
ip_editors AS (
    SELECT
        wiki,
        event.anonymous_user_token,
        COUNT(1) AS num_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] IN ('Mac OS X', 'iOS'), 1, NULL)) AS num_safari_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'Mac OS X', 1, NULL)) AS num_safari_macos_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'iOS', 1, NULL)) AS num_safari_ios_edits
    FROM event.editattemptstep
    WHERE event.action = 'saveSuccess'
    AND event.is_oversample = false -- Only use sampled events
    AND event.user_id = 0 -- Only IP edits
    GROUP BY wiki, event.anonymous_user_token
),
reg_agg AS (
    SELECT
        'registered' AS editor_class,
        COUNT(1) AS num_editors,
        COUNT(IF(num_safari_edits > 0, 1, NULL)) AS num_safari_editors,
        COUNT(IF(num_safari_macos_edits > 0
              AND num_safari_ios_edits = 0, 1, NULL)) AS num_safari_macos_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits = 0, 1, NULL)) AS num_safari_ios_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits > 0, 1, NULL)) AS num_safari_both_editors,
        SUM(num_edits) AS num_edits,
        SUM(num_safari_edits) AS num_safari_edits,
        SUM(num_safari_macos_edits) AS num_safari_macos_edits,
        SUM(num_safari_ios_edits) AS num_safari_ios_edits
    FROM reg_editors
),
ip_agg AS (
    SELECT
        'IP' AS editor_class,
        COUNT(1) AS num_editors,
        COUNT(IF(num_safari_edits > 0, 1, NULL)) AS num_safari_editors,
        COUNT(IF(num_safari_macos_edits > 0
              AND num_safari_ios_edits = 0, 1, NULL)) AS num_safari_macos_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits = 0, 1, NULL)) AS num_safari_ios_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits > 0, 1, NULL)) AS num_safari_both_editors,
        SUM(num_edits) AS num_edits,
        SUM(num_safari_edits) AS num_safari_edits,
        SUM(num_safari_macos_edits) AS num_safari_macos_edits,
        SUM(num_safari_ios_edits) AS num_safari_ios_edits
    FROM ip_editors
)
SELECT
    *
FROM reg_agg
UNION ALL
SELECT
    *
FROM ip_agg
'''

In [56]:
edits_data = presto.run(edits_editors_query)

In [ ]:
edits_data

In [58]:
edits_data.to_csv('overall_editors_edits_safari.csv')

In [75]:
wiki_edits_editors_query = '''
WITH reg_editors AS (
    SELECT
        wiki,
        event.user_id,
        COUNT(1) AS num_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] IN ('Mac OS X', 'iOS'), 1, NULL)) AS num_safari_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'Mac OS X', 1, NULL)) AS num_safari_macos_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'iOS', 1, NULL)) AS num_safari_ios_edits
    FROM event.editattemptstep
    WHERE event.action = 'saveSuccess'
    AND event.is_oversample = false -- Only use sampled events
    AND event.user_id != 0 -- No IP edits
    AND event.user_class IS NULL -- Set to "bot" if a bot user
    GROUP BY wiki, event.user_id
),
ip_editors AS (
    SELECT
        wiki,
        event.anonymous_user_token,
        COUNT(1) AS num_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] IN ('Mac OS X', 'iOS'), 1, NULL)) AS num_safari_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'Mac OS X', 1, NULL)) AS num_safari_macos_edits,
        COUNT(IF(regexp_like(user_agent_map['browser_family'],'Safari')
                 AND user_agent_map['os_family'] = 'iOS', 1, NULL)) AS num_safari_ios_edits
    FROM event.editattemptstep
    WHERE event.action = 'saveSuccess'
    AND event.is_oversample = false -- Only use sampled events
    AND event.user_id = 0 -- Only IP edits
    GROUP BY wiki, event.anonymous_user_token
),
reg_agg AS (
    SELECT
        wiki,
        'registered' AS editor_class,
        COUNT(1) AS num_editors,
        COUNT(IF(num_safari_edits > 0, 1, NULL)) AS num_safari_editors,
        COUNT(IF(num_safari_macos_edits > 0
              AND num_safari_ios_edits = 0, 1, NULL)) AS num_safari_macos_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits = 0, 1, NULL)) AS num_safari_ios_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits > 0, 1, NULL)) AS num_safari_both_editors,
        SUM(num_edits) AS num_edits,
        SUM(num_safari_edits) AS num_safari_edits,
        SUM(num_safari_macos_edits) AS num_safari_macos_edits,
        SUM(num_safari_ios_edits) AS num_safari_ios_edits
    FROM reg_editors
    GROUP BY wiki
),
ip_agg AS (
    SELECT
        wiki,
        'IP' AS editor_class,
        COUNT(1) AS num_editors,
        COUNT(IF(num_safari_edits > 0, 1, NULL)) AS num_safari_editors,
        COUNT(IF(num_safari_macos_edits > 0
              AND num_safari_ios_edits = 0, 1, NULL)) AS num_safari_macos_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits = 0, 1, NULL)) AS num_safari_ios_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits > 0, 1, NULL)) AS num_safari_both_editors,
        SUM(num_edits) AS num_edits,
        SUM(num_safari_edits) AS num_safari_edits,
        SUM(num_safari_macos_edits) AS num_safari_macos_edits,
        SUM(num_safari_ios_edits) AS num_safari_ios_edits
    FROM ip_editors
    GROUP BY wiki
),
wiki_names AS (
    SELECT
        database_code AS wiki,
        database_group AS project_group,
        english_name
    FROM canonical_data.wikis
),
aggs AS (
    SELECT
        *
    FROM reg_agg
    UNION ALL
    SELECT
        *
    FROM ip_agg
)
SELECT
    aggs.*,
    wiki_names.project_group,
    wiki_names.english_name
FROM aggs
JOIN wiki_names
ON aggs.wiki = wiki_names.wiki
'''

In [76]:
wiki_edits_data = presto.run(wiki_edits_editors_query)

In [ ]:
wiki_edits_data.loc[wiki_edits_data['wiki'] == 'nowiki']

In [78]:
wiki_edits_data.to_csv('wiki_editors_edits_safari.csv')

## Investigating the Anonymous User Token

When is it available, when isn't it?

In [79]:
token_query = '''
SELECT
    wiki,
    COUNT(1) AS num_edits,
    COUNT(event.anonymous_user_token) AS num_token_not_null
FROM event.editattemptstep
WHERE event.action = 'saveSuccess'
AND event.is_oversample = false -- Only use sampled events
AND event.user_id = 0 -- Only IP edits
GROUP BY wiki
'''

In [80]:
token_data = presto.run(token_query)

In [82]:
token_data['percent_not_null'] = 100.0 * token_data['num_token_not_null'] / token_data['num_edits']

In [ ]:
token_data.sort_values('percent_not_null', ascending = False).head(25)

Well, it's clear that this token isn't reliable at all. That's because it's only set for IP users who are part of the VE mobile default A/B test, ref [T225209](https://phabricator.wikimedia.org/T225209).


What other options do we have? Can we query `wmf_raw.mediawiki_private_cu_changes` and combine it with revision data to identify IP edits?

`cuc_this_oldid` reflects the revision ID. We'll join this with `wmf_raw.mediawiki_revision`. We then need to join `rev_actor` with `actor_id` in `wmf_raw.mediawiki_private_actor`. `actor_user` should be `NULL` for IP edits, and `actor_name` is then the IP address of the user. We should be able to join `cuc_ip` with `actor_name` as well, since both should be the IP address.

`cu_changes` only contains data from the given month. This means that we only have data for August 2021.

Note: `cuc_user` and `cuc_user_text` probably gives us this for free, so there's not a need to pull in data from `actor` and `revision`. Particularly being able to skip `revision` means we're sifting through a lot less data, but that'll have to be for next time.

In [120]:
ip_editors_query = '''
ADD JAR hdfs:///wmf/refinery/current/artifacts/refinery-hive-shaded.jar;
CREATE TEMPORARY FUNCTION ua AS 'org.wikimedia.analytics.refinery.hive.GetUAPropertiesUDF';

WITH ip_edits AS (
    SELECT
        rev.wiki_db,
        rev_id,
        rev_timestamp,
        actor_name
    FROM wmf_raw.mediawiki_revision AS rev
    JOIN wmf_raw.mediawiki_private_actor AS act
    ON rev.wiki_db = act.wiki_db
    AND rev.rev_actor = act.actor_id
    WHERE rev.snapshot = "2021-08"
    AND act.snapshot = "2021-08"
    AND rev_timestamp >= "20210801000000"
    AND rev_timestamp < "20210901000000"
    AND act.actor_user IS NULL
),
safari_edits AS (
    SELECT
        wiki_db,
        cuc_this_oldid,
        cuc_ip,
        ua(cuc_agent)['os_family'] AS os_family
    FROM wmf_raw.mediawiki_private_cu_changes
    WHERE month = "2021-08"
    AND ua(cuc_agent)['browser_family'] REGEXP "Safari"
),
ip_editors AS (
    SELECT
        ip_edits.wiki_db,
        ip_edits.actor_name,
        COUNT(1) AS num_edits,
        COUNT(IF(safari_edits.os_family IS NOT NULL
                 AND safari_edits.os_family IN ('Mac OS X', 'iOS'), 1, NULL)) AS num_safari_edits,
        COUNT(IF(safari_edits.os_family IS NOT NULL
                 AND safari_edits.os_family = "Mac OS X", 1, NULL)) AS num_safari_macos_edits,
        COUNT(IF(safari_edits.os_family IS NOT NULL
                 AND safari_edits.os_family = "iOS", 1, NULL)) AS num_safari_ios_edits
    FROM ip_edits
    LEFT JOIN safari_edits
    ON ip_edits.wiki_db = safari_edits.wiki_db
    AND ip_edits.rev_id = safari_edits.cuc_this_oldid
    AND ip_edits.actor_name = safari_edits.cuc_ip
    GROUP BY ip_edits.wiki_db, ip_edits.actor_name
),
ip_agg AS (
    SELECT
        wiki_db AS wiki,
        'IP' AS editor_class,
        COUNT(1) AS num_editors,
        COUNT(IF(num_safari_edits > 0, 1, NULL)) AS num_safari_editors,
        COUNT(IF(num_safari_macos_edits > 0
              AND num_safari_ios_edits = 0, 1, NULL)) AS num_safari_macos_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits = 0, 1, NULL)) AS num_safari_ios_editors,
        COUNT(IF(num_safari_ios_edits > 0
              AND num_safari_macos_edits > 0, 1, NULL)) AS num_safari_both_editors
    FROM ip_editors
    GROUP BY wiki_db
),
wiki_names AS (
    SELECT
        database_code AS wiki,
        database_group AS project_group,
        english_name
    FROM canonical_data.wikis
)
SELECT
    ip_agg.*,
    wiki_names.project_group,
    wiki_names.english_name
FROM ip_agg
JOIN wiki_names
ON ip_agg.wiki = wiki_names.wiki
'''

In [121]:
ip_editor_counts = hive.run(ip_editors_query)

In [ ]:
ip_editor_counts.loc[ip_editor_counts['wiki'] == 'enwiki']

In [123]:
ip_editor_counts.to_csv('ip_editor_counts_by_wiki.csv')

## Safari-exclusive Editors

In [102]:
safari_exclusive_editors_query = '''
WITH reg_editors AS (
    SELECT
        wiki,
        event.user_id,
        COUNT(1) AS num_edits,
        COUNT(IF(user_agent_map['browser_family'] = 'Safari', 1, NULL)) AS num_safari_edits,
        COUNT(IF(user_agent_map['browser_family'] = 'Mobile Safari', 1, NULL)) AS num_mob_safari_edits
    FROM event.editattemptstep
    WHERE event.action = 'saveSuccess'
    AND event.is_oversample = false -- Only use sampled events
    AND event.user_id != 0 -- No IP edits
    AND event.user_class IS NULL -- Set to "bot" if a bot user
    GROUP BY wiki, event.user_id
),
reg_agg AS (
    SELECT
        wiki,
        'registered' AS editor_class,
        COUNT(1) AS num_editors,
        COUNT(IF(num_edits = num_safari_edits, 1, NULL)) AS num_safari_ex_editors,
        COUNT(IF(num_edits = num_mob_safari_edits, 1, NULL)) AS num_mobile_safari_ex_editors
    FROM reg_editors
    GROUP BY wiki
),
wiki_names AS (
    SELECT
        database_code AS wiki,
        database_group AS project_group,
        english_name
    FROM canonical_data.wikis
)
SELECT
    reg_agg.*,
    wiki_names.project_group,
    wiki_names.english_name
FROM reg_agg
JOIN wiki_names
ON reg_agg.wiki = wiki_names.wiki
'''

In [103]:
safari_exclusive_editors = presto.run(safari_exclusive_editors_query)

In [ ]:
safari_exclusive_editors.loc[safari_exclusive_editors['wiki'] == 'enwiki']

In [106]:
safari_exclusive_editors.to_csv('safari_exclusive_editors.csv')

## Editing and IP Blocks

Note: this part of the analysis refers to [T292106](https://phabricator.wikimedia.org/T292106), and is our first attempt at answering questions for that task. We've since updated that task to clarify the questions, and there are references to specific notebooks for answering those.

We want to know how many non-bot edits have been blocked for belonging to one of the IP ranges that are being used by the Relay Service. To do this, we first take `wmf_raw.mediawiki_private_cu_changes` and modify it so that it has a boolean field for whether the IP address used to edit it was a relay address. We store this as a temporary table, then use that for querying.

Note that Hive JARs are also available in Spark, so we can use Spark for all of this.

This code is based on Martin Urbanec's code to [check pageviews from the Relay Service](https://github.com/urbanecm/2021-icloud-private-relay-usage/blob/master/check_private_relay_usage.py)

There are four places that IPs show up and we need to use `ipaddress` functionality:

1. Determine if an IP address used for editing was a relay.
2. Determine if an IP address used for editing is within either a local or a global block.
3. Determine if a local block is of a Relay IP or range.
4. Determine if a global block is of a Relay IP or range.

I'm also thinking that trying to do this with a combination of Spark and SQL is not the way to go. Since we need the functionality to identify IPs that got blocked, we might as well just do it all in Spark?

**Important note:** We're *only interested* in the number of non-bot edits that have been blocked, and whether it was a registered or non-registered user. We have data on the number of edits in other analysis. So, this is all about figuring out No. 4 in the list above and aggregating that count on a daily basis.

Sketch of how to do this:

1. Label edits in `mediawiki_private_cu_changes` with whether they were made from a relay IP or not.
2. Create a temp view of that dataset.
3. Write a query that extends that dataset with `rev_timestamp`, and whether the edit was a registered user or not. That query filters out bots by group membership or bot-like user name. Execute that query so it creates a Spark DataFrame on the cluster (`spark_session.sql(query)`).
4. Create a DataFrame of local blocks.
5. Cross-join the block DF with the edit DF and filter out blocks not made within 6 hours of the edits.
6. Iterate over the joined DF and label the block as whether it blocked the IP or not.
7. Repeat steps 4-6 for global blocks.
8. Union the two, remove duplicate edits, label them by whether they were blocked locally, globally, or both.
9. Aggregate across date, wiki_db, platform (Safair or Mobile Safari in our case), OS, user registration status.

**Notes:**

For future reference, there are more than 325,000 entries in the list of IP ranges. The code below does a linear search through that list for every IP or range it's testing. There must be more efficient methods of organizing that data to speed up the search. When testing on the command line, I noticed that range-based blocks are particularly slow. Part of that might be because the `ipaddress` library isn't performant, the other part is the fact that there are 325k networks to look through. *Update:* When it comes to improving the lookup algorithm, see [this notebook](https://github.com/wikimedia-research/2021-apple-relay-analysis/blob/main/T292106-relay-pageviews.ipynb) for that.

Reading up on how to determine network overlap, one Stack Overflow thread I found was [Check if two CIDR addresses intersect?](https://stackoverflow.com/questions/17206679/check-if-two-cidr-addresses-intersect) As [one response](https://stackoverflow.com/a/17206898) there points out "for the ranges to overlap, the first range's lowest address must be less than or equal to the second range's highest address, and vice versa." That means that we can do some organization of the data in order to determine that more quickly. This has been left for future work, e.g. if we need to determine how much of the network is blocked.

Another observation: the most likely key limiting factor here are the relay edits themselves. How many of them are there, across the entire dataset? We're only interested in log actions that were made within 6 hours of any edit, meaning that it might be faster to first identify all the relay edits, then query the local and global logging databases in MariaDB to grab the blocks and check if they match because those queries are limited by `log_timestamp`, `log_action`, and `log_type` so they're indexed queries.

Note: we continue to run into problems with identifying relays at scale. I've attempted to mitigate the problem by applying the SQL scheduler approach of "push filter statements as high up in the schedule as possible" in order to reduce the size of the initial dataset. This means that we're assuming for now that *all relay edits are made with Safari*, and filter `mediawiki_raw_cu_changes` to rows with Safari or Mobile Safari as the browser family. *Update:* The above mentioned notebook with an updated lookup algorithm solves this problem, we're aggregating tens of millions of pageviews in a few minutes.

In [8]:
# load iCloud's private relay egress ranges
# data comes from https://mask-api.icloud.com/egress-ip-ranges.csv
relay_ranges = pd.read_csv('datasets/egress-ip-ranges.csv',
                           sep=',', names=['range', 'country', 'region', 'city', 'empty']).drop(columns=['empty'])


In [9]:
binary_relay_nets = []
for net_raw in relay_ranges.range:
    net = ipaddress.ip_network(net_raw)
    binary_relay_nets.append((
        # network address in binary
        int(net.network_address),
        # netmask in binary
        int(net.netmask)
    ))

In [10]:
relay_nets = [ipaddress.ip_network(net_raw) for net_raw in relay_ranges.range]

In [11]:
def is_ip_private_relay(ip_raw):
    # this is based on https://stackoverflow.com/a/1004527
    ip = int(ipaddress.ip_address(ip_raw))
    for net in binary_relay_nets:
        if (ip & net[1]) == net[0]:
            return True
    return False

In [73]:
def is_private_relay_block(ip_or_range):
    try:
        ip = int(ipaddress.ip_address(ip_or_range))
        for net in binary_relay_nets:
            if (ip & net[1]) == net[0]:
                return(True)
    except ValueError:
        try:
            range = ipaddress.ip_network(ip_or_range)
            for net in relay_nets:
                if range.overlaps(net):
                    return(True)
        except ValueError:
            return(False)
    
    return(False)


In [62]:
def ip_is_blocked(raw_ip, block_ip_user_or_range):
    try:
        blocked_ip = ipaddress.ip_address(block_ip_user_or_range)
        return(raw_ip == block_ip_user_or_range) # exact IP blocked, do they match?
    except ValueError:
        try:
            ip = ipaddress.ip_address(raw_ip)
            blocked_range = ipaddress.ip_network(block_ip_user_or_range)
            return(ip in blocked_range) # is the IP in the blocked range?
        except ValueError:
            return(False) # not an IP address or network
    
    return(False)
        

In [18]:
spark_session.sql("ADD JAR hdfs:///wmf/refinery/current/artifacts/refinery-hive-shaded.jar")

DataFrame[result: int]

In [19]:
spark_session.sql("CREATE TEMPORARY FUNCTION ua AS 'org.wikimedia.analytics.refinery.hive.GetUAPropertiesUDF'")

DataFrame[]

In [16]:
safari_edits_query = '''
SELECT
    wiki_db,
    cuc_this_oldid,
    cuc_ip,
    ua(cuc_agent)['browser_family'] AS browser_family,
    ua(cuc_agent)['os_family'] AS os_family,
    ua(cuc_agent)['os_major'] AS os_major,
    ua(cuc_agent)['os_minor'] AS os_minor
FROM wmf_raw.mediawiki_private_cu_changes
WHERE ua(cuc_agent)['browser_family'] REGEXP "Safari"
'''

In [20]:
safari_edits_df = spark_session.sql(safari_edits_query)

In [21]:
cu_changes_data_is_relay = spark_session.createDataFrame(
    safari_edits_df.rdd.map(lambda r: T.Row(
        wiki_db = r.wiki_db,
        cuc_this_oldid = r.cuc_this_oldid,
        cuc_ip = r.cuc_ip,
        browser_family = r.browser_family,
        os_family = r.os_family,
        os_major = r.os_major,
        os_minor = r.os_minor,
        is_relay=is_ip_private_relay(r.cuc_ip)
    ))
)

In [22]:
relay_edits = cu_changes_data_is_relay.where(F.col("is_relay") == True).toPandas()

**Note:** We could've pulled the user ID and user name from `mediawiki_private_cu_changes` as well.

**Note:** `cuc_type` refers to `rc_type` in `recentchanges` and for many actions in `cu_changes` there's not an associated revision. E.g. autocreated users, abuse filter triggers, page protection. In that case, `cu_this_oldid` is 0, and so we need to filter those out. In other words, we can further limit the dataset by requiring `cu_this_oldid` to be non-zero.

In [35]:
len(relay_edits.loc[relay_edits['cuc_this_oldid'] != 0])

997

In [ ]:
(relay_edits.loc[relay_edits['cuc_this_oldid'] != 0].groupby('wiki_db')
 .agg({'cuc_this_oldid' : 'count'})
 .rename(columns = {'cuc_this_oldid' : 'n_edits'})
 .sort_values('n_edits', ascending = False)
 .head(20))

This is a number of edits that we can easily work with. So, we'll loop over them and use the `revision`, `archive`, `actor`, and `user_group` tables in MediaWiki to get revision information (timestamp, logged in/out, bot by group and user name), and then use the `logging` table to identify possible local and global blocks of the IP.

In [101]:
def add_rev_info(row):
    '''
    For the given row with data about a revision and IP used for editing, look up revision information
    and identify if the user was registered, a bot by group, or bot by name.
    '''
    
    ## bot user name regex from Data Engineering
    botUsernamePattern = r"^.*[Bb]ot([^a-zA-Z].*$|$)"
    
    ## Query to look up the revision timestamp, actor information
    rev_query = '''
    WITH bots AS (
        SELECT
            ug_user
        FROM user_groups
        WHERE ug_group = "bot"
    )
    SELECT
        rev_id,
        actor_user,
        actor_name,
        rev_timestamp,
        IF(actor_user IS NOT NULL, 1, 0) AS rev_user_is_registered,
        IF(ug_user IS NOT NULL, 1, 0) AS rev_user_is_bot_by_group,
        IF(actor_user IS NOT NULL
           AND actor_name REGEXP "{bot_regex}", 1, 0) AS rev_user_is_bot_by_name
    FROM revision
    JOIN revision_actor_temp
    ON rev_id = revactor_rev
    JOIN actor
    ON revactor_actor = actor_id
    LEFT JOIN bots
    ON actor_user = ug_user
    WHERE rev_id = {rev_id}    
    '''
    
    archive_query = '''
    WITH bots AS (
        SELECT
            ug_user
        FROM user_groups
        WHERE ug_group = "bot"
    )
    SELECT
        ar_rev_id AS rev_id,
        actor_user,
        actor_name,
        ar_timestamp AS rev_timestamp,
        IF(actor_user IS NOT NULL, 1, 0) AS rev_user_is_registered,
        IF(ug_user IS NOT NULL, 1, 0) AS rev_user_is_bot_by_group,
        IF(actor_user IS NOT NULL
           AND actor_name REGEXP "{bot_regex}", 1, 0) AS rev_user_is_bot_by_name
    FROM archive
    JOIN actor
    ON ar_actor = actor_id
    LEFT JOIN bots
    ON actor_user = ug_user
    WHERE ar_rev_id = {rev_id}
    '''
   
    rev_df = mariadb.run(rev_query.format(
        rev_id = row['cuc_this_oldid'],
        bot_regex = botUsernamePattern), row['wiki_db'])

    if len(rev_df) > 0:
        return(row.append(rev_df.squeeze()))
        
    ar_df = mariadb.run(archive_query.format(
        rev_id = row['cuc_this_oldid'],
        bot_regex = botUsernamePattern), row['wiki_db'])
        
    if len(ar_df) > 0:
        return(row.append(ar_df.squeeze()))
               
    return(pd.Series())

In [76]:
def add_local_blocks(row):
    '''
    For the given row with data about a revision and IP used for editing, look up potential blocks
    made within 6 hours of the revision, and identify if any of those blocked the given IP.
    '''
    
    local_block_query = '''
    SELECT
        log_timestamp,
        log_title
    FROM logging
    WHERE log_type = "block"
    AND log_action = "block"
    AND log_timestamp > "{rev_timestamp}"
    AND unix_timestamp(log_timestamp) - unix_timestamp("{rev_timestamp}") < 60 * 60 * 6    
    '''

    is_local_blocked = False
    
    block_data = mariadb.run(local_block_query.format(
        rev_timestamp = row['rev_timestamp']
    ), row['wiki_db'])
    
    for log_row in block_data.itertuples():
        if ip_is_blocked(row['cuc_ip'], log_row.log_title):
            is_local_blocked = True
    
    row['is_local_blocked'] = is_local_blocked
    
    return(row)
    

In [88]:
def add_global_blocks(row):
    '''
    For the given row with data about a revision and IP used for editing, look up potential
    global blocks on metawiki made within 6 hours of the revision, and identify if any of those
    blocked the given IP.
    '''
    
    global_block_query = '''
    SELECT
        log_timestamp,
        log_title
    FROM logging
    WHERE log_type = "gblblock"
    AND log_action = "gblock2"
    AND log_timestamp > "{rev_timestamp}"
    AND unix_timestamp(log_timestamp) - unix_timestamp("{rev_timestamp}") < 60 * 60 * 6    
    '''

    is_global_blocked = False
    
    block_data = mariadb.run(global_block_query.format(
        rev_timestamp = row['rev_timestamp']
    ), 'metawiki')
    
    for log_row in block_data.itertuples():
        if ip_is_blocked(row['cuc_ip'], log_row.log_title):
            is_global_blocked = True
    
    row['is_global_blocked'] = is_global_blocked
    
    return(row)
    

In [ ]:
relay_edits.loc[relay_edits['cuc_this_oldid'] != 0].head()

In [102]:
relay_full_df = relay_edits.loc[relay_edits['cuc_this_oldid'] != 0].apply(add_rev_info, axis = 1)

In [ ]:
relay_full_df.loc[relay_full_df['rev_user_is_bot_by_group'] == 1]

In [106]:
relay_full_df = relay_full_df.apply(add_local_blocks, axis = 1)

In [109]:
relay_full_df = relay_full_df.apply(add_global_blocks, axis = 1)

In [ ]:
relay_full_df.loc[relay_full_df['is_local_blocked'] == True]

In [ ]:
relay_full_df.loc[relay_full_df['is_global_blocked'] == True]